# Sklearn

Научимся делать разбиение набора данных на подвыборки. С помощью модуля **Sklearn.cross_validation** научимся делать разовые разбиения данных на обучение и тест, а также рассмотрим несколько часто встречающихся стратегий кросс-валидации.

## sklearn.model_selection

документация: http://scikit-learn.org/stable/modules/cross_validation.html

In [1]:
from sklearn import model_selection, datasets  # импортируем нужные модули

import numpy as np

### Разовое разбиение данных на обучение и тест с помощью train_test_split

In [2]:
iris = datasets.load_iris() # загрузим dataset ирисы Фишера

При решении задачи классификации нужно разбить данные на обучение и тест, чтобы впоследствии обучить модель на обучающей выборке и оценить ее качество на тесте. Для этого **cross_validation** предоставляет нам очень удобную функцию под названием **train_test_split**. Она позволяет нам построить разовое разбиение данных на обучение и тест. В качестве аргумента функция принимает набор данных, которые мы хотим разбить, набор меток классов, и также ей можно указать соотношение, в котором мы хотим разбивать данные. В данном случае отправляем 30 % объектов в тестовую выборку и все остальные — в обучающую. В результате получим 4 объекта, train_data и test_data — это части нашей выборки для обучения и для теста, непосредственное описание объектов, и train_labels и test_labels — это метки объектов из обучающей и тестовой выборки соответственно.

In [3]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(iris.data, 
                                                                                    iris.target, 
                                                                                    test_size = 0.3)

In [4]:
#убедимся, что тестовая выборка действительно составляет 0.3 от всех данных
float(len(test_labels))/len(iris.data)

0.3

In [9]:
# Оценим размер train_data или test_data и также можно оценить размер train_labels или test_labels. 
# Понятно, что размеры должны получиться одинаковые. 
print('Размер обучающей выборки: {} объектов \nРазмер тестовой выборки: {} объектов'.format(len(train_data),
                                                                                            len(test_data)))

Размер обучающей выборки: 105 объектов 
Размер тестовой выборки: 45 объектов


In [10]:
# выведем часть нашей обучающей и тестовой 
# выборки и посмотрим, как они выглядят
print('Обучающая выборка:\n', train_data[:5],'\n')
print('Тестовая выборка:\n', test_data[:5])

Обучающая выборка:
 [[7.2 3.  5.8 1.6]
 [6.  2.2 4.  1. ]
 [5.8 4.  1.2 0.2]
 [5.6 2.9 3.6 1.3]
 [5.5 4.2 1.4 0.2]] 

Тестовая выборка:
 [[6.4 3.1 5.5 1.8]
 [6.5 3.  5.2 2. ]
 [6.1 2.6 5.6 1.4]
 [5.1 2.5 3.  1.1]
 [6.  2.9 4.5 1.5]]


Выведем полностью список лейблов на обучении и на тесте и убедиться, что, действительно, это те же самые лейблы (метки от 0 до 2), и в обучении и в тесте присутствуют объекты всех классов

In [12]:
print('Метки классов на обучающей выборке:\n', train_labels, '\n')
print('Метки классов на тестовой выборке:\n', test_labels)

Метки классов на обучающей выборке:
 [2 1 0 1 0 0 2 1 0 2 0 2 1 0 1 2 1 0 0 0 2 0 0 0 1 1 0 0 1 1 1 1 2 2 2 2 0
 1 2 2 0 0 0 0 1 1 2 1 1 0 0 2 0 1 1 2 0 2 2 0 0 2 2 1 1 2 2 1 2 1 0 0 2 2
 1 1 2 2 1 2 0 2 1 1 1 1 2 0 1 0 0 0 2 2 1 2 2 1 0 0 2 2 2 2 2] 

Метки классов на тестовой выборке:
 [2 2 2 1 1 0 0 0 1 0 2 2 2 2 1 1 1 0 0 2 1 1 0 2 0 0 1 2 1 2 1 0 0 1 0 1 0
 1 1 0 1 1 2 0 0]


### Стратегии проведения кросс-валидации

Функция **train_test_split** позволяет строить разовое разбиение на обучение и тест. Часто это полезно, если мы просто хотим оценить качество нашей модели в одной точке. Если же мы хотим получить более строгую оценку, нам нужно пользоваться **кросс-валидацией**. 

In [13]:
# сгенерируем короткое подобие датасета, где элементы 
# совпадают с порядковым номером
X = range(0,10)

#### KFold

Давайте начнем с простого. Первая стратегия кросс-валидации, которую мы рассмотрим, называется **KFold**. В случае **KFold** мы разбиваем нашу выборку на K групп, при этом каждая из групп 1 раз участвует в тестировании и (K − 1) раз участвует в обучении. Для того чтобы получить такую кросс-валидацию, нам нужно использовать функцию **KFold**. В качестве аргументов она принимает на вход количество объектов, которое мы хотим разбивать, и количество фолдов, которое нам нужно. В отличие от функции train_test_split, функция **KFold** не строит нам разбиение исходных данных. Она возвращает нам пару индексов: индексы из обучения и индексы из тестов, с помощью которых мы далее можем самостоятельно разбить нашу выборку.

Применим эту функцию в случае, когда мы хотим разбить 10 объектов на 5 фолдов, и посмотрим, как выглядят наши индексы. 

In [16]:
kf = model_selection.KFold(n_splits = 5)
for train_indices, test_indices in kf.split(X):
    print(train_indices, test_indices)

[2 3 4 5 6 7 8 9] [0 1]
[0 1 4 5 6 7 8 9] [2 3]
[0 1 2 3 6 7 8 9] [4 5]
[0 1 2 3 4 5 8 9] [6 7]
[0 1 2 3 4 5 6 7] [8 9]


Вот мы видим наши разбиения. Каждый фолд получился размером 2 объекта, и поэтому обучающая выборка состоит каждый раз из 8 объектов, тестовая — из 2-х. Кстати, мы видим, что наши объекты разбиты по порядку. Если посмотреть на тестовую выборку, то мы видим, что каждый раз у нас индексы расположены в исходном порядке. Не всегда это удобно, в качестве примера можно вспомнить dataset ирисы Фишера, в котором объекты были исходно отсортированы по метке класса. В данном случае может получиться не очень хорошая ситуация, когда у нас в обучении или в тесте присутствуют представители не всех классов. Чтобы такого избежать, нам нужно указать параметр *shuffle* и сказать, что он должен быть равен *True*, то есть мы хотим сделать *shuffle* с нашей выборкой, хотим перемешать элементы. Вот давайте посмотрим, как будут выглядеть индексы.

In [43]:
kf = model_selection.KFold(n_splits = 2, shuffle = True)
for train_indices, test_indices in kf.split(X):
    print(train_indices, test_indices)

[3 5 6 8 9] [0 1 2 4 7]
[0 1 2 4 7] [3 5 6 8 9]


В данном случае мы построили разбиение на 2 фолда, чтобы было немножечко покороче, и видим, что в данном случае объекты уже расположены не по порядку. Но при этом если мы вызовем эту функцию еще раз или еще раз, мы получаем каждый раз разные разбиения. Не всегда это удобно. Если мы хотим, чтобы результат работы нашей функции был детерминированным, то нам нужно указать параметр *random_state*, зафиксировать какое именно случайное разбиение мы будем использовать. Вот давайте его укажем и посмотрим. 

In [44]:
kf = model_selection.KFold(n_splits = 2, shuffle = True, random_state = 1)
for train_indices, test_indices in kf.split(X):
    print(train_indices, test_indices)

[1 3 5 7 8] [0 2 4 6 9]
[0 2 4 6 9] [1 3 5 7 8]


Получили некоторое разбиение, повторные запуски этой функции приводят к таким же разбиениям. Часто это удобно, если мы хотим, чтобы результаты нашей работы были воспроизводимы.

#### StratifiedKFold

Следующая стратегия кросс-валидации, которую мы рассмотрим, называется **StratifiedKFold**. Она очень похожа на предыдущую, но есть существенное отличие. В данном случае мы сохраняем соотношение классов в обучающих и тестовых подвыборках. Для того чтобы запустить такую функцию, нам нужно передать ей не только количество объектов, но и непосредственно метки классов на объектах, так как разбиение происходит с учетом меток.

In [31]:
# Создадим набор меток классов. 
# В данном случае мы создадим список из 10 элементов, 
# первые 5 элементов будут равны 0, последние 5 элементов будут равны 1. 
# Таким образом мы получили задачу бинарной классификации.
y = np.array([0] * 5 + [1] * 5)
print(y)

# Сделаем разбиение на 2 фолда, также укажем параметр shuffle = True и random_state
skf = model_selection.StratifiedKFold(n_splits = 2, shuffle = True, random_state = 0)
for train_indices, test_indices in skf.split(X, y):
    print(train_indices, test_indices)

[0 0 0 0 0 1 1 1 1 1]
[3 4 6 8 9] [0 1 2 5 7]
[0 1 2 5 7] [3 4 6 8 9]


Видим, что метки получились такие, как мы хотели. Убедимся, что у нас соотношение объектов в обучающих и тестовых выборках также 50 на 50, как и в исходных метках. Видим, что объекты с индексом 3 и 4 имеют метку 0, объекты с индексом 8 и 9 имеют метку 1. Соответственно, соотношение правильное. 

Для примера давайте создадим другой список меток, где метки уже будут идти через одну (0 1 0 1) и посмотрим, как изменятся наши индексы. Функцию запускаем практически с такими же параметрами. Вот видим, что, да, действительно, индексы другие, и видим, что вот наш индекс 1 и 5 — это объекты с меткой 1, индексы 2 и 8 — объекты с меткой 0, то есть соотношение опять правильное.

In [33]:
target = np.array([0, 1] * 5)
print(target)

skf = model_selection.StratifiedKFold(n_splits = 2,shuffle = True)
for train_indices, test_indices in skf.split(X, target):
    print(train_indices, test_indices)

[0 1 0 1 0 1 0 1 0 1]
[0 1 4 5 7] [2 3 6 8 9]
[2 3 6 8 9] [0 1 4 5 7]


#### ShuffleSplit

Следующая интересная стратегия — это **ShuffleSplit**. Она позволяет строить так называемые случайные перестановки. Таким образом мы можем получить очень много выборок, при этом мы можем специфицировать размер обучающей выборки, и у нас нет никаких ограничений на то, сколько раз каждый объект должен появиться в обучении или в тесте. Каждый раз мы действуем с возвращением, то есть мы получаем одно разбиение и дальше можем строить другое независимо от предыдущего. В качестве аргументов функции нужно указать количество наших объектов, сколько итераций мы хотим и размер тестовой выборки. Вот давайте разобъем данные в соотношении: 80 % — обучение и 20 % — тест, и посмотрим, как будут выглядеть наши выборки. 

In [34]:
ss = model_selection.ShuffleSplit(n_splits = 10, test_size = 0.2)

for train_indices, test_indices in ss.split(X):
    print(train_indices, test_indices)

[0 6 4 5 2 1 8 9] [3 7]
[8 0 1 5 2 7 4 3] [6 9]
[7 8 4 1 5 3 6 2] [0 9]
[1 3 9 7 4 8 6 5] [0 2]
[2 8 6 9 7 4 5 0] [3 1]
[5 7 4 3 9 8 2 6] [0 1]
[3 4 0 5 8 7 9 2] [6 1]
[4 9 8 7 3 6 2 1] [5 0]
[4 9 2 6 7 0 1 5] [8 3]
[8 3 4 2 7 1 0 6] [9 5]


Ну вот мы построили 10 итераций. Невооруженным взглядом видно, что у нас нет никаких ограничений на то, сколько раз объект должен встретиться в обучении или в тесте. Вот, в частности, мы видим, что объект с меткой 0 сразу несколько раз попадает в тест, разбиение действительно случайное. 

#### StratifiedShuffleSplit

Но **shuffle_split** также можно стратифицировать. Для этого нужно использовать функцию **StratifiedShuffleSplit**, и в этом случае мы тоже будем получать выборки, которые имеют исходное соотношение классов. Опять же, для этого нам придется передать *target* и целевую метку в функцию. И давайте посмотрим, как это выглядит. Сделаем поменьше итераций, сделаем 4 и убедимся, что все правильно. 

In [35]:
target = np.array([0] * 5 + [1] * 5)
print(target)

sss = model_selection.StratifiedShuffleSplit(n_splits = 4, test_size = 0.2)
for train_indices, test_indices in sss.split(X, target):
    print(train_indices, test_indices)

[0 0 0 0 0 1 1 1 1 1]
[6 7 5 2 1 4 0 8] [3 9]
[6 4 1 2 5 7 3 8] [0 9]
[2 5 1 0 4 7 8 6] [3 9]
[2 4 8 7 1 9 5 3] [0 6]


Вот, действительно, каждый раз мы видим, что в тесте есть один объект нулевого класса, один объект первого класса, таким образом, очевидно, что и в обучении у нас также получилась сбалансированная выборка.

#### Leave-One-Out

Стратегия LeaveOneOut позволяет оставить каждый объект в тесте 1 раз. Таким образом, тестовая выборка всегда состоит из одного объекта, и каждый объект из нашего набора данных 1 раз присутствует в тесте. Это очень удобная стратегия кросс-валидации, которую хорошо использовать в случае, когда мы имеем небольшую выборку данных. Давайте посмотрим, как это выглядит. Функции достаточно передать только количество объектов, и легко проверить, что мы получили правильное разбиение. Каждый объект 1 раз присутствует в тестовой выборке.

In [36]:
loo = model_selection.LeaveOneOut()

for train_indices, test_index in loo.split(X):
    print(train_indices, test_index)

[1 2 3 4 5 6 7 8 9] [0]
[0 2 3 4 5 6 7 8 9] [1]
[0 1 3 4 5 6 7 8 9] [2]
[0 1 2 4 5 6 7 8 9] [3]
[0 1 2 3 5 6 7 8 9] [4]
[0 1 2 3 4 6 7 8 9] [5]
[0 1 2 3 4 5 7 8 9] [6]
[0 1 2 3 4 5 6 8 9] [7]
[0 1 2 3 4 5 6 7 9] [8]
[0 1 2 3 4 5 6 7 8] [9]


Больше стратегий проведения кросс-валидации доступно здесь: http://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators